In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 8.2 MB 18.7 MB/s 
     |████████████████████████████████| 371 kB 56.9 MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=854548 sha256=600a02e99b8f26c7ab12e6732c7033533c8b96a5ae176a5cb032695439f2d66c
  Stored in directory: /root/.cache/pip/wheels/85/51/a4/2de41ff197786537075027c27b479a38da92f50abc86634445
Successfully built DAWG


In [ ]:
import random
import re
import string
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import ToTensor
import warnings 
from sklearn.linear_model import LogisticRegression
from PIL import Image
from IPython.display import clear_output
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from typing import List
from sklearn.metrics import classification_report
import pickle
from typing import List
import nltk
import string
import pymorphy2
import codecs

In [ ]:
warnings.filterwarnings(action='ignore',category=UserWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning, module='gensim')  
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='gensim')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='smart_open') 
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='sklearn')
warnings.filterwarnings(action='ignore',category=DeprecationWarning, module='scipy')    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/tenderhack/dataset_base.xlsx')
data.head()

,Идентификатор СТЕ,Наименование СТЕ,Категория,Описание,Характеристики СТЕ,Регионы поставки,Кол-во заключенных контрактов,Поставщики,Страна происхождения,Другая продукция в контрактах,Просмотры,Идентификатор КПГЗ,Код КПГЗ,Модель,Цена
0,20528973,"Светодиодная лента 2835, ULTRA, 12 Вольт, 60 l...",Лампы светодиодные,NaN,"[{""Name"":""Цвет свечения"",""Id"":-506425117,""Valu...","[{""Name"":""Москва""}]",NaN,"[{""SupplierId"":2310783,""Name"":""ООО \""ЛЮМЕНВАТТ...",NaN,NaN,1,12106442.0,01.10.05.02.04,2835ULTRA,NaN
1,34860341,Светодиодный светильник Geniled ЛПО 1200?180?...,Настенно-потолочные светильники,NaN,"[{""Name"":""Вид климатического исполнения"",""Id"":...","[{""Name"":""Свердловская""},{""Name"":""Москва""}]",1.0,"[{""SupplierId"":1559525,""Name"":""Общество с огра...",РОССИЯ,NaN,41,12114082.0,01.10.05.03.01.05,ЛПО 1200?180?40 40Вт 5000К Опал,NaN
2,34172198,Елка пристенно-угловая Графская премиум 210 см...,Предметы внутреннего интерьера,NaN,"[{""Name"":""Цвет"",""Id"":340085190,""Value"":""Зелены...","[{""Name"":""Москва""},{""Name"":""Все регионы""}]",NaN,"[{""SupplierId"":1348988,""Name"":""ИП Чепало Оксан...",NaN,NaN,5,68093.0,01.20.08,Елка пристенно-угловая Графская премиум 210 см.,NaN
3,1318014,Картридж HP CE278A/Canon 728 LJ P1560/1566/160...,Расходные материалы и комплектующие для лазерн...,NaN,"[{""Name"":""Тип расходника"",""Id"":317455168,""Valu...","[{""Name"":""Москва""},{""Name"":""Московская""}]",9.0,"[{""SupplierId"":1111141,""Name"":""Общество с огра...",NaN,"[{""OtherSkuId"":1398583,""OtherSkuName"":""013R006...",6,12105204.0,01.13.13.01.01,SF-CE278(2K),NaN
4,1251411,Штамп самонаборный Pr.C20/3-Set 38х14 (аналог ...,Штемпели,NaN,"[{""Name"":""Наличие рамки"",""Id"":366100137,""Value...","[{""Name"":""Белгородская""},{""Name"":""Марий Эл""},{...",71.0,"[{""SupplierId"":1100019,""Name"":""ООО \""ЦТО\"""",""I...",АВСТРИЯ,"[{""OtherSkuId"":1171023,""OtherSkuName"":""Berling...",72,12120460.0,01.15.06.07,Штамп самонаборный пласт. 3стр. Pr.C20/3-Set 3...,"[{""Cost"":454.67000},{""Cost"":620.00000},{""Cost""..."


In [ ]:
len(data)

200000

In [ ]:
data['Категория'].value_counts()

Расходные материалы и комплектующие для лазерных принтеров и МФУ                                  6519
Ручки канцелярские                                                                                3230
Одежда специальная для защиты от общих производственных загрязнений и механических воздействий    3015
Папки пластиковые                                                                                 1994
Учебники печатные общеобразовательного назначения                                                 1951
                                                                                                  ... 
Масла моторные авиационные                                                                           1
Аксессуары к аппарату "eva"                                                                          1
Грузы для скелетного вытяжения                                                                       1
Бриллиантовый крезиловый синий раствор ивд                               

In [ ]:
data.dtypes.value_counts()

object     11
float64     2
int64       2
dtype: int64

In [ ]:
type(data['Характеристики СТЕ'][0])

str

In [ ]:
import json
characts = []
for i in data['Характеристики СТЕ']:
  try:
    characts.append(json.loads(i))
  except:
    pass


In [ ]:
print(characts[0])

[{'Name': 'Цвет свечения', 'Id': -506425117, 'Value': 'Холодный'}, {'Name': 'Мощность', 'Id': -506424125, 'Value': '7.00000', 'Unit': 'Вт'}, {'Name': 'Напряжение', 'Id': -506424124, 'Value': '12', 'Unit': 'В'}, {'Name': 'Яркость', 'Id': -506425116, 'Value': '760', 'Unit': 'лм'}, {'Name': 'Плотность led на метр', 'Id': -506425119, 'Value': '300', 'Unit': 'шт'}, {'Name': 'Размер светодиодов', 'Id': -506425118, 'Value': '283', 'Unit': 'smd'}]


In [ ]:
all_characts = {}
charact_counter = {}
for charact_set in characts:
  for charact in charact_set:
    name = charact['Name'].strip().lower()
    if name[0] == '"':
      name = name[1:-1]
    elif name[0] == "'":
      name = name[1:]
    if name not in all_characts:
      all_characts[name] = set()
      charact_counter[name] = 0
    charact_counter[name]+=1
    try:
      all_characts[name].add(charact['Value'].strip().lower())
    except KeyError:
      pass


In [ ]:
print(len(all_characts))
print(list(all_characts.keys())[:20])

57717
['цвет свечения', 'мощность', 'напряжение', 'яркость', 'плотность led на метр', 'размер светодиодов', 'вид климатического исполнения', 'вид рассеивателя', 'коэффициент мощности', 'вид товаров', 'вид энергетического оборудования', 'габаритные размеры (шхгхв)', 'световой поток', 'цветовая температура', 'гарантия', 'степень защиты', 'цвет', 'металлическая подставка', 'материал', 'вид ели']


In [ ]:
print(list(sorted(all_characts.keys()))[:20])

['+частота процессора', '- заряд', '- разряд', '. рн 10% водного раствора мыла', '1  – пропанол', '1 disilet с отслаиваемой оболочкой', '1 xro силиконовый катетер  4,2 fr', '1 замес', '1 порт', '1 радиотрубка', '1 туннельная игла', '1,62', '1-й класс', '1-й тип коннектора', '10/100base-tx с поддержкой poe', '100 внутренних абонентов. расширяется до 200 с помощью dsp модуля d3', '100% арабика', '10base-t / 100base-tx / 1000base-t', '12 модулей размером', '12. способ измерения температуры']


In [ ]:
good_keys = [key for key in charact_counter if charact_counter[key]>=10]
len(good_keys)

10555

In [ ]:
sorted(good_keys)[:20]

['100% арабика',
 '3 в 1',
 '3d эффект',
 '3d-датчик ускорения',
 '3d-печать',
 '3g',
 '4g',
 '4g lte',
 '[in] номинальный ток',
 'access control list',
 'adf (податчик оригиналов)',
 'aoh',
 'atx',
 'av выход',
 'bec',
 'bio-средство',
 'bios',
 'bluetooth',
 'bluetooth встроенный',
 'classless inter-domain routing (cidr)']

In [ ]:
for i in sorted(good_keys)[:20]:
  print(i, all_characts[i])

100% арабика {'да', 'нет'}
3 в 1 {'да', 'нет'}
3d эффект {'эффект движения', 'объемная'}
3d-датчик ускорения {'да', 'нет'}
3d-печать {'объмная с движением', 'объемная'}
3g {'да', 'нет'}
4g {'да', 'нет'}
4g lte {'да', 'нет'}
[in] номинальный ток {'80', '63', '32', '160', '10', '16', '125', '250', '315', '1', '7', '40', '95', '25', '9', '100', '6', '2'}
access control list {'нет', 'поддерживается, до 50 списков доступа'}
adf (податчик оригиналов) {'есть', 'нет', 'двусторонний', 'наличие', 'обычный', '1', 'односторонний', 'да', 'dadf  на 50 листов'}
aoh {'есть', 'да'}
atx {'да', 'нет'}
av выход {'да', 'нет'}
bec {'40.00000', '18.00000', '3.60000', '4.00000', '20.00000', '16.00000', '15', '10.00000', '25.00000', '8.00000', '30.00000', '15.00000', '52.00000', '3.00000', '5.00000', '1.00000'}
bio-средство {'да', 'нет'}
bios {'efi ami bios, 128 мбит', '128 mb flash rom, uefi ami bios', 'ami', 'efi ami, 64 мбит x2 с поддержкой dualbios', 'efi ami bios, 2x 128 мбит', 'efi ami bios, 1 x128 мбит'

In [ ]:
data_dict = {}
for iterr, st in data.iterrows():
  data_dict[st['Идентификатор СТЕ']] = st

In [ ]:
connected = {}
for iterr, st in data.iterrows():
  if st['Категория'] not in connected:
    connected[st['Категория']] = {}
  if not pd.isna(st['Другая продукция в контрактах']) and len(st['Другая продукция в контрактах'].strip())>0:
    st_others = st['Другая продукция в контрактах']
    edge_inda = -st_others[::-1].index("}")
    if edge_inda!=0:
      st_others = st_others[:edge_inda]+']'
    else:
      st_others += ']'
    # print(st_others[-20:])
    st_data = json.loads(st_others)
    for prod in st_data:
      try:
        connected_cat = data_dict[prod['OtherSkuId']]['Категория']
        if connected_cat not in connected[st['Категория']]:
          connected[st['Категория']][connected_cat] = 0
        connected[st['Категория']][connected_cat] += 1
      except KeyError:
        pass

      

In [ ]:
print(len(connected))

3810


In [ ]:
list(connected.keys())[:10]

['Лампы светодиодные',
 'Настенно-потолочные светильники',
 'Предметы внутреннего интерьера',
 'Расходные материалы и комплектующие для лазерных принтеров и МФУ',
 'Штемпели',
 'Блокнот',
 'Знаки различия наплечные, нарукавные, кокарды, нашивки',
 'Грамоты',
 'Салфетки, насадки для уборки, полотна технические, ветошь',
 'Флаги, вымпелы, прочая флажная продукция']

In [ ]:
connected['Блокнот']

{'Data-кабели': 7,
 'USB-накопители твердотельные (флеш-драйвы)': 23,
 'Автомобильная косметика': 2,
 'Аксессуары для банка и инкассации': 56,
 'Аксессуары для кофеварок и кофемашин': 1,
 'Альбомы для рисования': 20,
 'Альбомы для фотографий': 1,
 'Антисептики кожные': 10,
 'Аптечки медицинские': 1,
 'Баннерные декорации': 2,
 'Батареи аккумуляторные никель-металл-гибридные': 8,
 'Бейджи': 151,
 'Бинты марлевые медицинские': 4,
 'Бирки для ключей': 53,
 'Бланк бухгалтерский': 43,
 'Бланк самокопирующий': 1,
 'Бланки (кроме бухгалтерских)': 46,
 'Блоки бумажные для записей': 3011,
 'Блокнот': 2667,
 'Боксы для бумаг пластиковые': 26,
 'Брошюраторы (кроме брошюраторов полиграфических)': 15,
 'Булавки': 72,
 'Бумага для офисной техники белая': 251,
 'Бумага для офисной техники цветная': 612,
 'Бумага для плоттеров и копировальных работ': 26,
 'Бумага для рисования': 41,
 'Бумага для черчения (ватман)': 146,
 'Бумага копировальная': 10,
 'Бумага мелованная для печати': 6,
 'Бумага миллимет

In [62]:
connected_keys = list(connected.keys())
num_cat = enumerate(connected_keys)
decoder = {category:i for i, category in num_cat}
print(decoder['Блокнот'])

5
